In [1]:
from objects import EnvironmentManager
from helpers import *

env = EnvironmentManager()
env.start_vehicles()
env.route.load_all_pax()

observation, reward, terminated, truncated, info = env.step(action=None)
while not terminated:
    action = get_action('RA')
    observation, reward, terminated, truncated, info = env.step(action=action)

history = env.get_history()
# for key in history:
#     history[key]['scenario'] = scenario
#     history[key]['episode'] = i
#     results[key].append(history[key])

In [35]:
from rl_env import FlexSimEnv
import numpy as np

In [36]:
rl_env  = FlexSimEnv()

In [46]:
rl_env.reset()

({'control_stop_idx': array([0], dtype=int32),
  'n_requests': array([1], dtype=int32),
  'headway': array([563.], dtype=float32),
  'schedule_deviation': array([-56.], dtype=float32)},
 {'lost_requests': 0,
  'off_schedule_trips': 0,
  'total_trips': 0,
  'time': 706.0,
  'veh_idx': 1,
  'direction': 'out'})

In [59]:
rl_env.step(action=1)

({'control_stop_idx': array([3], dtype=int32),
  'n_requests': array([1], dtype=int32),
  'headway': array([506.], dtype=float32),
  'schedule_deviation': array([-33.], dtype=float32)},
 0.0,
 0,
 0,
 {'lost_requests': 0,
  'off_schedule_trips': 0,
  'total_trips': 0,
  'time': 3877.0,
  'veh_idx': 1,
  'direction': 'in'})

In [1]:
from params import REWARD_WEIGHTS
REWARD_WEIGHTS

{'lost_requests': -1.0, 'off_schedule_trips': -1.0}

In [1]:
# train
from training_claude import train_vehicles

agent, scores = train_vehicles(reward_weights={'off_schedule_trips': -2.5, 'lost_requests': -1.0})

Training with shared policy...
Episode 0/500, Avg Score: -2.42, Epsilon: 1.0000
Episode 20/500, Avg Score: -2.06, Epsilon: 0.8302
Episode 40/500, Avg Score: -1.82, Epsilon: 0.6865
Episode 60/500, Avg Score: -2.33, Epsilon: 0.5688
Episode 80/500, Avg Score: -2.59, Epsilon: 0.4788
Episode 100/500, Avg Score: -2.42, Epsilon: 0.3999
Episode 120/500, Avg Score: -2.53, Epsilon: 0.3384
Episode 140/500, Avg Score: -2.27, Epsilon: 0.2866
Episode 160/500, Avg Score: -2.19, Epsilon: 0.2418
Episode 180/500, Avg Score: -2.60, Epsilon: 0.2044
Episode 200/500, Avg Score: -2.65, Epsilon: 0.1752
Episode 220/500, Avg Score: -2.47, Epsilon: 0.1482
Episode 240/500, Avg Score: -2.71, Epsilon: 0.1264
Episode 260/500, Avg Score: -2.27, Epsilon: 0.1069
Episode 280/500, Avg Score: -2.55, Epsilon: 0.0917
Episode 300/500, Avg Score: -2.84, Epsilon: 0.0783
Episode 320/500, Avg Score: -2.40, Epsilon: 0.0674
Episode 340/500, Avg Score: -2.89, Epsilon: 0.0578
Episode 360/500, Avg Score: -2.19, Epsilon: 0.0500
Episod

In [22]:
from training_claude import load_agent
import numpy as np

# Example of loading and using the model
loaded_agent = load_agent('shared_dqn_agent.pth')

# Example of getting an action from a state
# The state should be a dictionary with the expected keys
state = {
    "control_stop_idx": np.array([3]),      # Make sure this is an array
    "n_requests": np.array([2]),            # Make sure this is an array
    "headway": np.array([600.5]),            # Make sure this is an array
    "schedule_deviation": np.array([400.2])  # Make sure this is an array
}
vehicle_idx = 0  # Example vehicle index

# Get action from loaded model
action = loaded_agent.act(state, veh_idx=vehicle_idx, eval_mode=True)
print(f"For state {state}, the model recommends action: {action}")

For state {'control_stop_idx': array([3]), 'n_requests': array([2]), 'headway': array([600.5]), 'schedule_deviation': array([400.2])}, the model recommends action: 0


In [31]:
from rl_env import FlexSimEnv

In [32]:
recorded_actions = []
recorded_observations = []
rl_env = FlexSimEnv()

observation, info = rl_env.reset()
vehicle_idx = info['veh_idx']
action = loaded_agent.act(observation, vehicle_idx, eval_mode=True)

observation, reward, terminated, truncated, info = rl_env.step(action=action)
while not terminated:
    # Use the loaded agent to predict the action based on current observation
    action = loaded_agent.act(observation, vehicle_idx, eval_mode=True)
    recorded_actions.append(action)
    recorded_observations.append(observation)
    observation, reward, terminated, truncated, info = rl_env.step(action=action)

In [34]:
for item in rl_env.env.route.inter_event:
    print(item)

{'lost_requests': 0, 'off_schedule_trips': 2, 'total_trips': 0}
{'lost_requests': 3, 'off_schedule_trips': 8, 'total_trips': 0}


In [28]:
zero_idxs = [i for i in range(len(recorded_actions)) if recorded_actions[i] == 0]

In [29]:
zero_idxs

[]

In [25]:
recorded_observations[25]

{'control_stop_idx': array([2], dtype=int32),
 'n_requests': array([1], dtype=int32),
 'headway': array([1109.], dtype=float32),
 'schedule_deviation': array([541.], dtype=float32)}

In [38]:
recorded_observations[21]

{'control_stop_idx': array([3], dtype=int32),
 'n_requests': array([1], dtype=int32),
 'headway': array([1319.], dtype=float32),
 'schedule_deviation': array([761.], dtype=float32)}

In [37]:
recorded_observations[19]

{'control_stop_idx': array([1], dtype=int32),
 'n_requests': array([2], dtype=int32),
 'headway': array([1149.], dtype=float32),
 'schedule_deviation': array([647.], dtype=float32)}

In [ ]:
rl_env = FlexSimEnv()

observation, info = rl_env.reset()
action = model.predict(observation, deterministic=True)
action = int(action[0])
observation, reward, terminated, truncated, info = rl_env.step(action=action)
while not terminated:
    # Use the loaded agent to predict the action based on current observation
    action = model.predict(observation, deterministic=True)
    action = int(action[0])
    observation, reward, terminated, truncated, info = rl_env.step(action=action)

history = rl_env.env.get_history()
for key in history:
    history[key]['scenario'] = 'RL_' + model_name
    history[key]['episode'] = i
    results[key].append(history[key])